In [2]:
# finds average card length just for funsies!
train_data = np.array(np.load('training_data.npy',allow_pickle=True))
if True:
    total_length = 0
    for card in train_data:
        total_length +=  int(len(card[0].split()))
print(total_length/len(train_data))


866.9346192665695


In [12]:
# finds max and min (non-zero) card length also for funsies
train_data = np.array(np.load('training_data.npy',allow_pickle=True))
max_length = 0
min_length = 10000000 # yeah should've used int max or wtv but i already know the max length
min_dex = 0
if True:
    for index,card in enumerate(train_data):
        if max_length < int(len(card[0].split())):
            max_length = int(len(card[0].split()))
        if min_length > int(len(card[0].split())) and int(len(card[0].split()))!=0:
            min_length = int(len(card[0].split()))
            min_dex = index
print('Max length is: ' + str(max_length))
print('Min length is: ' + str(min_length))
print('Min card contents: ' + str(train_data[min_dex][0]))
print(min_dex)

Max length is: 59677
Min length is: 1
Min card contents: https://open.spotify.com/track/4v55Nqito6jGwkGFOCZZ42?si=f63901d7eebf490f 
9871


In [7]:
# total number of cards just to keep count
print(len(np.load('training_data.npy',allow_pickle=True)))

35668


In [2]:
# boilerplate stuff 

# imports! 
import numpy as np
from transformers import BertTokenizerFast,BertForSequenceClassification,TFBertModel
from transformers import Trainer, TrainingArguments
from transformers import ReformerTokenizer,ReformerModel
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
from accelerate import Accelerator
accelerator = Accelerator()
# GPU meow 
#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# loads the data 
train_data = np.array(np.load('training_data.npy',allow_pickle=True))
X = list(train_data[:,0])
y = list(train_data[:,1])
model_name ="bert-base-uncased"
#model_name = 'bert-large-uncased'
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)
nlp = TFBertModel.from_pretrained(model_name)
def convert_card_to_word_vectors(card_text,classification_values,use_bert_base=True):
    # this ws just easier 
    #sample_text = X[card_index]
    #class_values = y[card_index]
    sample_text = card_text
    class_values = classification_values
    if use_bert_base:
        model_name ="bert-base-uncased"
    else:
        model_name = 'bert-large-uncased'
    ##tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)
    
    input_ids = np.array(tokenizer.encode(sample_text,add_special_tokens=False,is_split_into_words=True,truncation=True))[None,:]
    input_ids_printable = input_ids[0]
    # i wrote this next part of code while watching like three very drunk people watch "Tall Girl 2"
    # so it's probably not going to be particularly robust or efficent 
    old_text = sample_text
    new_text = tokenizer.batch_decode(input_ids_printable)
    # this loop basically converts the old classification vector to be compatable with the tokens that BERT
    # needs to work. i hate this stupid method 
    oldex = 0
    newdex = 0
   
    token_mult_list = []
    new_class_list = []
    while oldex < len(old_text):
        tokens_per_word = 0
        while newdex<len(new_text) and (new_text[newdex].lower().lstrip('#') in old_text[oldex].lower()):
            #print('meow')
            tokens_per_word+=1
            newdex+=1
        token_mult_list.append(tokens_per_word)
        oldex+=1
    for count,value in enumerate(token_mult_list):
        index = 0
        while value>index:
            new_class_list.append(class_values[count])
            index+=1
    #print(token_mult_list)
    #print('Old Thing: ' + str(len(old_text)))
    #print('New Thing: ' + str(len(new_text)))
    #print('Mult Classes: ' + str(len(token_mult_list)))
    #print('New Classes: ' + str(len(new_class_list)))


    #train_encodings = np.array(tokenizer(new_text, truncation=True, padding=True, max_length=1000,is_split_into_words=True))
    ##nlp = TFBertModel.from_pretrained(model_name)


    embedding = nlp(input_ids)
    return_embedding = embedding[0][0]
    #print(embedding[0][0])
    if(len(new_class_list)==return_embedding.shape[0]):
        return new_class_list,return_embedding
    else:
        return None,None
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    # thanks to Greenstick on StackOverflow for the method 
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [24]:
# example use of the conver_to_word_vectors method

card_index = 30

print(convert_card_to_word_vectors(X[card_index].split(),y[card_index]))

([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 1, 1, 1, 4, 4, 3, 1, 1, 4, 4, 3, 4, 4, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 1, 1, 4, 2, 2, 2, 1, 3, 3, 3, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 4, 4, 4, 4, 4, 1, 1, 1, 3, 3, 3, 4, 4, 3, 1, 1, 1], <tf.Tensor: shape=(132, 768), dtype=float32, numpy=
array([[ 0.3447914 ,  0.38311923,  0.18867965, ..., -0.46555212,
         0.5244969 ,  0.16768585],
       [ 0.52054405,  0.00200855,  0.4149542 , ..., -0.59987897,
         0.8867201 , -0.21326253],
       [ 0.06089576, -0.2205686 ,  0.02235103, ..., -0.14053151,
         0.3026108 ,  0.17655082],
       ...,
       [ 1.2743224 , -0.19928378,  0.43570018, ..., -0.34392828,
        -0.11686718,  0.2048613 ],
       [ 0.16592826,  0.11902209,  0.35186273, ..., -0.12351574,
         0.01183689, -0.7323766 ],
       [ 0.52903754, -0.46968013,  0.

In [27]:
# just a place for whatever code I need for shit 
print(len(train_data))

35668


In [13]:

# so you want a list of info vectors of 768 length (X) and list of class highlighting values (y) length 1 each of course
# to get those for each card, divide into 512 (less? maybe ~450 safer) chunks and run above process on them, then for each
# resulting vomit of info vectors, classify using a neural network or whatever in pytorch
info_vectors = []
class_values = []
max_size = 450
i = 0
error_count = 0
data_length = len(train_data)
# IHC is a Mongolian CS:GO team which used to be called Checkmate, so this is like
# a joke about checkpoint saving I guess? look I'm not exactly very smart ok give me a fucking break
ihc_length = 3700 # should make like 10 files or so unless I fucked something up 
chunk_count = 1
ihc_counter = 0

for iter,data_pair in enumerate(train_data):
    if ihc_counter==ihc_length:
        np.save('info_vectors_' + str(chunk_count),info_vectors)
        np.save('class_values_' + str(chunk_count),class_values)
        ihc_counter=0
        chunk_count+=1
        info_vectors = []
        class_values = []
    else:
        ihc_counter+=1
    printProgressBar(iter,data_length)
    #print(1,end=' ')
    X_curr = data_pair[0].split()
    y_curr = data_pair[1]
    X_curr_split = [X_curr[i:i + max_size] for i in range(0, len(X_curr), max_size)]
    y_curr_split = [y_curr[i:i + max_size] for i in range(0, len(y_curr), max_size)]
    #print(len(X_curr_split))
    #print(len(y_curr_split))
    for count,chunk in enumerate(X_curr_split):
        #print(2,end=' ')
        new_class_list,new_word_vectors = convert_card_to_word_vectors(chunk,y_curr_split[count])
        if new_class_list is not None:
            for count_two, word_vect in enumerate(new_word_vectors):
                info_vectors.append(word_vect)
                class_values.append(new_class_list[count_two])
        else:
            error_count+=1
print(error_count)
np.save('info_vectors_-1',info_vectors)
np.save('class_values_-1',class_values)
        

4388█████████████████████████████████████████████████████████████████████████████████████████████████-| 100.0% 


In [ ]:
# pytorch imports and code and stuff 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
import os
PATH = './card_net.pth'

class CardNetSmall(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(3, 6, 5)
        self.conv2 = nn.Conv1d(6, 16, 5)
        self.fc1 = nn.Linear(512, 1000)
        self.fc2 = nn.Linear(1000, 1000)
        self.fc3 = nn.Linear(1000, 1000)
        self.fc4 = nn.Linear(1000, 700)
        self.fc5 = nn.Linear(700, 5)

    def forward(self, x):
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

class Card_Class_Data(Dataset):
    def __init__(self, word_data):
        self.word_data=word_data
        
    def __len__(self):
        return len(self.card_data)
    
    def __getitem__(self, index):
        """ img_name = os.path.join(self.img_path,self.img_data.loc[index, 'labels'],
                                self.img_data.loc[index, 'Images'])
        image = Image.open(img_name)
        #image = image.convert('RGB')
        image = image.resize((300,300))
        label = torch.tensor(self.img_data.loc[index, 'encoded_labels'])
        if self.transform is not None:
            image = self.transform(image)
        return image, label """
        return card_index[index]
train_loader = Card_Class_Data()#TODO add stuff here
net = CardNetSmall()
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), amsgrad=True) # hopefully if prof. cytskajfsaldfas's class taught me anything it's that ordinary SGD sucks 

In [ ]:
# does the training 

model = CardNetSmall()
#net.load_state_dict(torch.load(PATH))

#TODO get data in some configuration that doens't suck 
model, optimizer, data = accelerator.prepare(model, optimizer, data)


for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        #loss.backward()
        
        accelerator.backward(loss)
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

torch.save(net.state_dict(), PATH)